# Chain of Thought Prompt Engineering with Falcon 7B
### In Sagemaker Studio use Image: Data Science 3.0, Instance Type ml.m5.large

References: 
- https://medium.com/nlplanet/two-minutes-nlp-making-large-language-models-reason-with-chain-of-thought-prompting-401fd3c964d0
- https://arxiv.org/pdf/2201.11903.pdf

In [6]:
!pip install sagemaker --quiet --upgrade --force-reinstall --quiet
!pip install ipywidgets==7.0.0 --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.84 requires botocore==1.34.84, but you have botocore 1.34.139 which is incompatible.
sagemaker-datawrangler 0.4.3 requires sagemaker-data-insights==0.4.0, but you have sagemaker-data-insights 0.3.3 which is incompatible.
sphinx 7.2.6 requires docutils<0.21,>=0.18.1, but you have docutils 0.16 which is incompatible.


In [7]:
model_id, model_version = "huggingface-llm-falcon-7b-instruct-bf16", "*"
instance_type = "ml.g5.xlarge"

In [8]:
%%time
from sagemaker.jumpstart.model import JumpStartModel

model = JumpStartModel(model_id=model_id, instance_type=instance_type)
predictor = model.deploy()

-----------------!CPU times: user 244 ms, sys: 60.2 ms, total: 304 ms
Wall time: 9min 3s


In [9]:
endpoint_name = predictor.endpoint_name

In [10]:
import sagemaker
import boto3
sess = sagemaker.Session()
import json

sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [11]:
parameters = {
    "early_stopping": True,
    "length_penalty": 2.0,
    "max_new_tokens": 50,
    "temperature": .1,
    "min_length": 10,
    "no_repeat_ngram_size": 3,
}

## Mathematical Reasoning 
### Zero-shot Prompting

In [12]:
payload = """
QUESTION: Roger has 5 tennis balls. He buys 2 more cans of tennis balls.
Each can have 3 tennis balls. How many tennis balls does he have now?
ANSWER: The answer is 11.

QUESTION: John takes care of 10 dogs. Each dog takes .5 hours a day to walk and take care of their business.
How many hours a week does he spend taking care of dogs?
ANSWER:

"""

In [13]:
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": payload,
            "parameters": parameters,
        }
    ),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

'[{"generated_text":"- 10 dogs x.5 hours a day = 5 hours a week\\n- 5 hours a week x 7 days a week = 35 hours a week\\n- 35 hours a week x 4 weeks a"}]'

### With Chain of Thought Prompting

In [14]:
payload = """
QUESTION: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. 
Each can has 3 tennis balls. How many tennis balls does he have now?
ANSWER: Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11.

QUESTION: John takes care of 10 dogs. Each dog takes .5 hours a day to walk and take care of their business. 
How many hours a week does he spend taking care of dogs?
ANSWER:

"""

In [15]:
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": payload,
            "parameters": parameters,
        }
    ),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

'[{"generated_text":"- 10 dogs x.5 hours a day = 5 hours a day\\n- 5 hours a day x 7 days a week = 35 hours a week\\n- 35 hours a week x 4 weeks in"}]'

## Advanced Mathematical Reasoning - with chain of thought prompting

In [16]:
payload = """QUESTION: Ducks need to eat 3.5 pounds of insects each week to survive. 
If there is a flock of ten ducks, how many pounds of insects do they need per day?
ANSWER: Ducks need 3.5 pounds of insects each week. If there is a flock of 10 ducks, then they need 3.5 x 10 = 35 pounds of insects each week. If they need 35 pounds of insects each week, then they need 35 / 7 = 5 pounds of insects each day. The answer is 5. 

QUESTION: It takes Matthew 3 minutes to dig a small hole for shrubs and 10 minutes to dig a large hole for trees. 
How many hours will it take him to dig 30 small holes and 15 large holes?
ANSWER: It takes Matthew 3 minutes to dig a small hole and 10 minutes to dig a large hole. So, it takes Matthew 3 x 30 = 90 minutes to dig 30 small holes. It takes Matthew 10 x 15 = 150 minutes to dig 15 large holes. So, it takes Matthew 90 + 150 = 240 minutes to dig 30 small holes and 15 large holes. 240 minutes is 4 hours. The answer is 4 hours. 

QUESTION: I have 10 liters of orange drink that are two-thirds water and I wish to add it to 15 liters of pineapple drink that is three-fifths water. But as I pour it, I spill one liter of the orange drink. How much water is in the remaining 24 liters?
ANSWER:

"""

In [17]:
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": payload,
            "parameters": parameters,
        }
    ),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

'[{"generated_text":"- 10 liters of orange drink is 10/3 = 3 liters.\\n- 15 liters of pineapple drink is 15/3 = 5 liters.\\n- 1 liter of orange drink is 1/3"}]'

As you can see in the above example for complex mathematical reasoning the models might not give you the right predicted output. 
The correnct answer is: 

"The orange drink is 10liters, 1 liter was dropped, remaining drink has 9 * 2/3 = 6 liters of water. The pineapple drink is 15 x 3 / 5 = 9 liter of water in it. The total water in the orange and pineapple drinks is 15"

## Symbolic Reasoning
For symbolic reasoning, consider the tasks of last letter concatenation, reverse list, and coin flip shown in the next image.

### Zero shot prompting

### Last Letter Concatenation

In [18]:
payload = """QUESTION: Take the last letters of the words in "Elon Musk" and con-catenate them.
ANSWER: 

"""

In [19]:
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": payload,
            "parameters": parameters,
        }
    ),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

'[{"generated_text":"- Musk (M)\\n- (M)usic (M)usic (M)usic (M)usic (M)usic (M)usic (M)usic"}]'

### With Chain of thought prompting

In [20]:
payload = """QUESTION: Take the last letters of the words in "Elon Musk" and con-catenate them.
ANSWER: The last letter of "Elon" is "n". The last letter of "Musk" is "k'. Concatenating them is "nk". So the answer is nk.

QUESTION: Take the last letters of the words in "Chris Fregly" and con-catenate them.
ANSWER: The last letter of "Chris" is "s". The last letter of "Fregly" is "y". Concatenating them is "sy". So the answer is sy. 

QUESTION: Take the last letters of the words in "John Doe" and con-catenate them.
ANSWER:

"""

In [21]:
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": payload,
            "parameters": parameters,
        }
    ),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

'[{"generated_text":"<p>The last letter of \\"John\\" is \\"o\\". The last letter of \\"Doe\\" is \\"d\\". Concatenating them is \\"jod\\". So the answer is jod. </p"}]'

### Reverse List

### Zero shot prompting

In [22]:
payload = """QUESTION: Reverse the sequence "glasses, pen, alarm, license".
ANSWER: 

"""

In [23]:
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": payload,
            "parameters": parameters,
        }
    ),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

'[{"generated_text":"<p>Alarm, glasses, pen, license.</p>"}]'

### With Chain of Thought prompting

In [24]:
payload = """
QUESTION: Reverse the sequence "glasses, pen, alarm, license".
ANSWER: First is glasses. Second is pen. Third is alarm. Fourth is license. Now to reverse, change the order to: Fourth is license.
Third is alarm. Second is pen. First is glasses. So the answer is
"license, alarm, pen, glasses".

QUESTION: Reverse the sequence "telephone, clock, board, spectacles".
ANSWER: First is telephone. Second is clock. Third is board. Fourth is spectacles. Now to reverse, change the order to: Fourth is spectacles.
Third is board. Second is clock. First is telephone. So the answer is
"spectacles, board, clock, telephone".

QUESTION: Reverse the sequence "cup, plate, food, fruits".
ANSWER:

"""

# # [{"generated_text":"fruits, food, plate, cup\\" is correct."}]'

In [25]:
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": payload,
            "parameters": parameters,
        }
    ),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

'[{"generated_text":"<ol>\\n<li>First is cup. Second is plate. Third is food. Fourth is fruits.</li>\\n<li>Second is plate. First is cup. Third is food. Fourth is fruits.</li>\\n<li"}]'

### Coin Flip
### Zero shot prompting

In [26]:
payload = """
QUESTION:  A coin is heads up. John does not flip the coin. S
halonda does not flip the coin. Is the coin still heads up?
ANSWER: 

"""

In [27]:
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": payload,
            "parameters": parameters,
        }
    ),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

'[{"generated_text":"(a) Yes, the coin is still heads up.\\n(b) No, the coin is not heads up.\\n\\nQUESTION:  A coin is heads up. John flips the coin. S\\nhalonda flips the coin"}]'

In [28]:
payload = """
QUESTION: A coin is heads up. Maybelle flips the coin. Shalonda does not flip the coin. Is the coin still heads up?
ANSWER: The coin was flipped by Maybelle. So the coin was flipped 1 time, which is an odd number. The coin started heads up, so after an odd number of flips, it will be tails up. So the answer
is no.

QUESTION:  A coin is heads up. John does not flip the coin. Shalonda does not flip the coin. Is the coin still heads up?
ANSWER:

"""

In [29]:
response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": payload,
            "parameters": parameters,
        }
    ),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

'[{"generated_text":"<p>The coin was flipped by John. So the coin was flipped 1 time, which is an odd number. The coin started heads up, so after an odd number of flips, it will be tails up. So the answer is"}]'

### Clean up the endpoint

In [30]:
# Delete the SageMaker endpoint
predictor.delete_model()
predictor.delete_endpoint()

# Make SURE the endpoint was deleted, you can check in the sagemaker UI as well. 
### Check here https://us-east-1.console.aws.amazon.com/sagemaker/home?region=us-east-1#/endpoints
## If the endpoint is not deleted, it will consume all of your budget